In [1]:
import chardet

with open("data-unspsc-codes.csv", 'rb') as f:
    result = chardet.detect(f.read())
    print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [2]:
import pandas as pd

df = pd.read_csv("data-unspsc-codes.csv", encoding=result['encoding'])

df.head()

,Segment,Segment Name,Family,Family Name,Class,Class Name,Commodity,Commodity Name
0,10000000,Live Plant and Animal Material and Accessories...,10100000,Live animals,10101500,Livestock,10101501,Cats
1,10000000,Live Plant and Animal Material and Accessories...,10100000,Live animals,10101500,Livestock,10101502,Dogs
2,10000000,Live Plant and Animal Material and Accessories...,10100000,Live animals,10101500,Livestock,10101504,Mink
3,10000000,Live Plant and Animal Material and Accessories...,10100000,Live animals,10101500,Livestock,10101505,Rats
4,10000000,Live Plant and Animal Material and Accessories...,10100000,Live animals,10101500,Livestock,10101506,Horses


In [3]:
cat_cols = ["Segment Name", "Family Name", "Class Name", "Commodity Name"]

In [4]:
from itertools import combinations

for i, col in enumerate(cat_cols):
    unique = pd.unique(df[col])
    n_unqiue = len(unique)
        
    total = len(df[col])
    print(f"col: {col}\nunqiue: {n_unqiue}\ntotal: {total}\n")
    
    if i < len(cat_cols) - 1:
        next_branch_counts = []
        next_branches: dict[str, set] = {}
        
        for uc in unique:
            df_next = df[df[col] == uc]
            next_branch_counts.append(len(pd.unique(df_next[cat_cols[i+1]])))
            next_branches[uc] = set(pd.unique(df_next[cat_cols[i+1]]))
    
        print(f"next level:\nmax:{max(next_branch_counts)}\navg:{sum(next_branch_counts)/len(next_branch_counts)}\n")
        
        ambiguous = False
        for a, b in combinations(next_branches.keys(), 2):
            intersection = next_branches[a] & next_branches[b]
            if len(intersection) > 0:
                ambiguous = True
                print(f"The following nodes appear in both {a} and {b} for {cat_cols[i+1]}: {intersection}")
            
        if ambiguous:    
            print(f"{col} has ambiguous branches\n")
        else:
            print(f"{col} does NOT have ambiguous branches\n")
            

col: Segment Name
unqiue: 57
total: 71502

next level:
max:43
avg:8.157894736842104

Segment Name does NOT have ambiguous branches

col: Family Name
unqiue: 465
total: 71502

next level:
max:67
avg:11.425806451612903

Family Name does NOT have ambiguous branches

col: Class Name
unqiue: 5313
total: 71502

next level:
max:99
avg:13.45793337097685

Class Name does NOT have ambiguous branches

col: Commodity Name
unqiue: 71502
total: 71502



In [5]:
from typing import Callable, Union



class Node:
    """Class representing a node in the tree"""
    
    breadcrumb_name: Union[str, None] = None
    """The name of the breadcrumb found in the original dataset or None"""
    
    condition: Union[str, None] = None
    """The description or condition representing belonging to this node or None if this Node is the root"""
    
    extras: dict[str, any]
    """Extra information stored at this node for enrichment"""
    
    parent: Union['Node', None]
    """The parent Node or None if this Node is the root"""
    
    children: list['Node']
    """The child Nodes"""
    
    # TODO: create next condition when children are updated
    # TODO: children should be property
    
    def is_leaf(self) -> bool:
        return len(self.children) == 0
    
    def is_root(self) -> bool:
        return self.parent == None
    
    def is_from_breadcrumb(self) -> bool:
        return self.breadcrumb_name != None
    
    def __init__(self, condition: str = None, parent: 'Node' = None, extras: dict[str, any] = {}):
        self.condition = condition
        self.parent = parent
        self.extras = extras or {}
        self.children = []
        
    def add_children(self, children: list['Node']):
        self.children.extend(children)
        
        
def _create_partial_tree_from_breadcrumb(df: pd.DataFrame, parent: Node, idx: int, breadcrumb_cols: list[str], extra_cols_map: dict[str, list[str]] = None) -> list[Node]:
    breadcrumb_col = breadcrumb_cols[idx]
    breadcrumbs = pd.unique(df[breadcrumb_col])
    nodes = []
    
    for breadcrumb in breadcrumbs:
        extras = {}
        extra_cols = extra_cols_map.get(breadcrumb_col, []) if extra_cols_map else []
        sub_df = df[df[breadcrumb_col] == breadcrumb]
        for extra_col in extra_cols:
            extras[extra_col] = pd.unique(sub_df[extra_col])
        node = Node(condition=breadcrumb, parent=parent, extras=extras)
        if idx + 1 < len(breadcrumb_cols):
            node.add_children(_create_partial_tree_from_breadcrumb(sub_df, node, idx + 1, breadcrumb_cols=breadcrumb_cols, extra_cols_map=extra_cols_map))
        nodes.append(node)
    return nodes
        
        
    
    
def create_tree_from_breadcrumbs(df: pd.DataFrame, breadcrumb_cols: list[str], extra_cols_map: dict[str, list[str]] = None) -> Node:
    """_Create a tree from breadcrumbs left in the dataset. Suitable for datasets with an existing heirarchy. `breadcrumb_cols` is an ordered list of columns that represent heirarchical levels. `extra_cols_map` contains extra columns to store in `Node.extras` for each breadcrumb if applicable._

    
    Args:
        df (pd.DataFrame): dataset
        breadcrumb_cols (list[str]): ordered heirarchy columns
        extra_cols_map (dict[str, list[str]], optional): list of extra columns to enrich nodes for each breadcrumb column. Defaults to None.
    """
    
    root = Node()
    root.add_children(_create_partial_tree_from_breadcrumb(df, parent=root, idx=0, breadcrumb_cols=breadcrumb_cols, extra_cols_map=extra_cols_map))
    return root
    

In [6]:
root = create_tree_from_breadcrumbs(df, breadcrumb_cols=["Segment Name", "Family Name", "Class Name", "Commodity Name"], extra_cols_map={"Segment Name": ["Segment"], "Family Name": ["Family"], "Class Name": ["Class"], "Commodity Name": ["Commodity"]})

In [7]:
def _check_branches(nodes: list[Node]):
    from itertools import combinations


    n_sub_branches = 0
    max_sub_branches = 0
    all_children = []
    for node in nodes:
        n_sub_branches += len(node.children)
        all_children.extend(node.children)
        max_sub_branches = max(len(node.children), max_sub_branches)
    
    avg_sub_branches = n_sub_branches / len(nodes)
    
    print(f"sub_branches: {n_sub_branches}, avg: {avg_sub_branches}, max: {max_sub_branches}\n")
    
    if len(all_children) > 0:
        _check_branches(all_children)

def check_tree(root: Node):
    _check_branches(nodes=root.children)

In [8]:
check_tree(root)

sub_branches: 465, avg: 8.157894736842104, max: 43

sub_branches: 5313, avg: 11.425806451612903, max: 67

sub_branches: 71502, avg: 13.45793337097685, max: 99

sub_branches: 0, avg: 0.0, max: 0



In [9]:
import plotly.express as px

def display_tree(root: Node, max_depth=3):
    labels = []
    parents = []

    def traverse(node: Node, parent_label: str = ""):
        label = str(node.condition or "Unnamed")
        labels.append(label)
        parents.append(parent_label)

        for child in node.children:
            traverse(child, parent_label=label)

    traverse(root)

    fig = px.sunburst(
        names=labels,
        parents=parents,
        maxdepth=max_depth
    )
    fig.update_layout(margin=dict(t=10, l=10, r=10, b=10))
    fig.show()


In [10]:
def get_node_conditions(node: Node):
    n = node
    labels = []
    while n != None:
        if n.condition is not None:
            labels.append(n.condition)
        n = n.parent
        
    return list(reversed(labels))

def format_node(node: Node):
    labels = get_node_conditions(node=node)
    return " > ".join(labels)

In [11]:
node = root.children[0]

print(format_node(node))

display_tree(node, max_depth=2)

Live Plant and Animal Material and Accessories and Supplies


In [12]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
)

from langchain_core.vectorstores import InMemoryVectorStore

def create_vector_store(texts: list[str]) -> InMemoryVectorStore:
    vectorstore = InMemoryVectorStore.from_texts(
        texts,
        embedding=embeddings,
    )
    
    return vectorstore

In [13]:
cats = [n.condition for n in node.children]

vectorstore = create_vector_store(cats)

In [14]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

def sample_from_embeddings(vectorstore: InMemoryVectorStore, samples: int = 10) -> list[list[float]]:
    store = vectorstore.store
    rev_map = {}
    embeddings = []
    for idx, (k, v) in enumerate(store.items()):
        vector = v['vector']
        rev_map[idx] = v['text']
        embeddings.append(vector)
        
    normalized_embeddings = normalize(embeddings, norm='l2', axis=1)

    k = samples

    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(normalized_embeddings)

    from sklearn.metrics.pairwise import cosine_similarity

    representative_indices = []
    for center in kmeans.cluster_centers_:
        sims = cosine_similarity([center], normalized_embeddings)[0]
        idx = np.argmax(sims)
        representative_indices.append(idx)

    selected_keys = [rev_map[i] for i in representative_indices]
    return selected_keys

In [15]:
from langchain_ollama import ChatOllama
create_llm = lambda: ChatOllama(
    model="qwen2.5:14b",
    # temperature=0,
)

In [22]:
from typing import Tuple
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from langchain_community.callbacks import get_openai_callback


class TokenCounts(BaseModel):
    prompt: int
    completion: int
    total: int

class CategoryAnswer(BaseModel):
    categories: list[str] = Field(description="A list of categories that do not overlap. Must contain at least two categories.")

def ask_model_category(node: Node) -> Tuple[CategoryAnswer, TokenCounts]:
    vectorstore = create_vector_store([n.condition for n in node.children])
    selected_cats = sample_from_embeddings(vectorstore=vectorstore, samples=min(15, len(node.children)))
        
    template = """
    Your job is to create a set categories that will serve as nodes in a tree.
    The new nodes will lie between the parent category below and the child categories listed.
    
    Previous categories that describe these items:
    {prev_conditions}
    
    Direct parent category that should be divided:
    {prev_condition}
    
    Sample of items that should fit into the new categories:
    {conditions}

    The items in the full dataset cover the scope of: {scope}

    Provide two or more new categories to lie between the parent and children.
    These categories should not overlap and should serve to divide the existing children between new nodes to improve search performance.
    Only create categories that would have one or more item fit into them.
    The categories created should cover all items that would satisfy the previous categories.
   
    Your objective is to predict the kind of items that would be categorized using the previous categories and scope and provide a new set of categories to divide them into smaller portions.
    Do not specify catch-all categories like "Other" and "None".
    
    Do not create new categories that are ambigious with, duplicates of, or direct inverses of other categories listed above.
    """.strip()
    prev_condition = "All"
    if node.parent is not None:
        prev_condition = node.parent.condition
    prompt = ChatPromptTemplate.from_template(template)
    prompt = prompt.format(conditions="\n* ".join(selected_cats), prev_conditions="\n* ".join(get_node_conditions(node)), scope="Products across all industries", prev_condition=prev_condition)

    with get_openai_callback() as cb:
        llm = create_llm().with_structured_output(CategoryAnswer)
        response = llm.invoke([HumanMessage(prompt)])
        
    return response, TokenCounts(prompt=cb.prompt_tokens, completion=cb.completion_tokens, total=cb.total_tokens)

In [23]:
cats, tokens = ask_model_category(node=root)
cats

CategoryAnswer(categories=['Agriculture and Forestry Equipment', 'Healthcare and Medical Supplies', 'Educational Materials and Services', 'Construction and Building Components', 'Industrial Machinery and Handling', 'Consumer Goods and Personal Care', 'Energy and Natural Resources Services', 'Chemical Products and Biochemicals', 'Financial Tools and Insurance Solutions', 'Environmental Management and Consulting', 'Electrical Infrastructure and Lighting', 'Media and Communication Equipment'])

In [24]:
from typing import Optional


# class CategorizationItem:
#     description: str
#     extras: dict[str, any]
#     nodes: list[Node]
    
#     def __init__(self, description):
#         self.description = description
#         self.extras = {}
#         self.nodes = []
        
class CategoryChoice(BaseModel):
    category_number: int = Field(description="The number of the category chosen according to the numbers in the list.")
    
def categorize_next(item_description: str, root: Node) -> Tuple[Optional[Node], TokenCounts]:
    conditions = [n.condition for n in root.children]
    conditions.append("None")
    
    conditions = [f"{i+1}: {cond}" for i,cond in enumerate(conditions)]
    
    template = """
    With the given item and categories, determine which of the categories correctly describes the item.
    Choose only the number of the category which is correct, otherwise choose None from the list.
    
    Categories:
    {categories}
    
    Item:
    {item}
    """.strip()
    
    prompt = ChatPromptTemplate.from_template(template)
    prompt = prompt.format(categories={"\n* ".join(conditions)}, item=item_description)
    
    with get_openai_callback() as cb:
        llm = create_llm().with_structured_output(CategoryChoice)
        choice: CategoryChoice = llm.invoke([HumanMessage(prompt)])
    
    choice_node = None
    
    if choice.category_number - 1 < len(root.children):
        choice_node = root.children[choice.category_number - 1]
    
    return choice_node, TokenCounts(prompt=cb.prompt_tokens, completion=cb.completion_tokens, total=cb.total_tokens)
    

In [49]:
from ipywidgets import IntProgress, HBox, VBox, Label, HTML
from IPython.display import display, clear_output
import time


class ProgressBars:
    completion_tokens: int = 0
    prompt_tokens: int = 0
    total_tokens: int = 0
    
    progress_total: IntProgress = None
    progress_batch: IntProgress = None
    label_total_progress: HTML = None
    label_status: HTML = None
    label_tokens: HTML = None
    ui: VBox = None
    
    start_time: float = time.time()
    
    def __init__(self, n_leaves: int):
        self.progress_total = IntProgress(value=0, min=0, max=n_leaves, description='Leaves Completed:', bar_style='info', layout={'width': '90%'})
        self.label_status = HTML(value="<b>Status:</b> Starting...")
        self.label_tokens = HTML(value="Prompt: 0 | Completion: 0 | total: 0")
        self.label_total_progress = HTML(value=self._format_total_progress(n_leaves=0))
        self.ui = VBox([
            self.progress_total,
            self.label_total_progress,
            self.label_status,
            self.label_tokens
        ])

    def start_new_batch(self, n_children: int):
        self.progress_batch = IntProgress(value=0, min=0, max=n_children, description='Children Categorized:', bar_style='success', layout={'width': '90%'})
        self.ui.children = (self.progress_total, self.label_total_progress, self.progress_batch, self.label_status, self.label_tokens)
    
    def update_status(self, status: str):
        self.label_status.value = status
        
    def update_batch_progress(self, n_complete: int):
        self.update_total_progress(self.progress_total.value)
        if self.progress_batch is None:
            return
        self.progress_batch.value = n_complete
        
    def increment_tokens(self, counts: TokenCounts):
        self.completion_tokens += counts.completion
        self.prompt_tokens += counts.prompt
        self.total_tokens += counts.total
        self.label_tokens.value = f"Prompt: {self.prompt_tokens:,} | Completion: {self.completion_tokens:,} | Total: {self.total_tokens:,}"
        
    def _format_total_progress(self, n_leaves: int):
        elapsed = time.time() - self.start_time
        rate = (n_leaves) / elapsed if elapsed > 0 else 0
        eta = (self.progress_total.max - (n_leaves)) / rate if rate > 0 else 0
        def format_time(seconds):
            seconds = int(seconds)
            days, seconds = divmod(seconds, 86400)
            hours, seconds = divmod(seconds, 3600)
            minutes, seconds = divmod(seconds, 60)

            return f"{days:02}:{hours:02}:{minutes:02}:{seconds:02}"
        return f"{n_leaves}/{self.progress_total.max} | Elapsed: {format_time(elapsed)} | ETA: {format_time(eta)}"
        
    def update_total_progress(self, n_leaves: int):
        self.progress_total.value = n_leaves
        self.label_total_progress.value = self._format_total_progress(self.progress_total.value)

In [50]:
completed_leaves = 0

def optimize_tree(root: Node, max_children: int, progress_bars: ProgressBars = None) -> None:
    global completed_leaves
    if root.is_leaf():
        completed_leaves += 1
        if progress_bars is not None:
            progress_bars.update_total_progress(completed_leaves)
    while len(root.children) > max_children:
        print(f"Working on node '{root.condition}' with {len(root.children)} subcategories.")
        if progress_bars is not None:
            progress_bars.update_status(f"<b>Status:</b> Working on node '{root.condition}' with {len(root.children)} subcategories.") 
        new_cats, token_counts = ask_model_category(root)
        if progress_bars is not None:
            progress_bars.increment_tokens(token_counts)
        old_children = root.children
        root.children = [Node(condition=c) for c in new_cats.categories]
        uncategorized: list[Node] = []
        
        print(f"Created categories: {[c.condition for c in root.children]}")
        
        if progress_bars is not None:
            progress_bars.start_new_batch(len(old_children))
        n_categorized: int = 0
        for node in old_children:
            choice, token_counts = categorize_next(item_description=node.condition, root=root)
            n_categorized += 1
            if progress_bars is not None:
                progress_bars.update_batch_progress(n_categorized)
                progress_bars.increment_tokens(token_counts)
            if choice is None:
                uncategorized.append(node)
            else:
                choice.add_children([node])
                
        print(f"Rebalanced {len(old_children)-len(uncategorized)}/{len(old_children)} categories.")
                
        print(f"Failed to categorize: {[u.condition for u in uncategorized]}")
        
        root.add_children(uncategorized)
    
    
    for child in root.children:
        optimize_tree(root=child, max_children=max_children, progress_bars=progress_bars)

In [ ]:
root = create_tree_from_breadcrumbs(df, breadcrumb_cols=["Segment Name", "Family Name", "Class Name", "Commodity Name"], extra_cols_map={"Segment Name": ["Segment"], "Family Name": ["Family"], "Class Name": ["Class"], "Commodity Name": ["Commodity"]})
progress_bars = ProgressBars(n_leaves=len(df))
display(progress_bars.ui)
optimize_tree(root=root, max_children=25, progress_bars=progress_bars)

Working on node 'None' with 57 subcategories.
Created categories: ['Agriculture and Forestry', 'Construction and Building Materials', 'Healthcare and Medical Equipment', 'Consumer Goods and Services', 'Energy and Mining Services', 'Chemical Products and Biochemicals', 'Financial Instruments and Insurance Solutions', 'Environmental Protection and Remediation', 'Information Technology Infrastructure', 'Education and Professional Training']
Rebalanced 48/57 categories.
Failed to categorize: ['Defense and Law Enforcement and Security and Safety Equipment and Supplies', 'Sports and Recreational Equipment and Supplies and Accessories', 'Published Products', 'Industrial Cleaning Services', 'Transportation and Storage and Mail Services', 'Public Utilities and Public Sector Related Services', 'National Defense and Public Order and Security and Safety Services', 'Politics and Civic Affairs Services', 'Organizations and Clubs']
Working on node 'Live Plant and Animal Material and Accessories and S

In [67]:
display_tree(root, max_depth=3)

In [59]:
root.children[0].children